# Deconvolve simulated data with linear functions, vary number of samples

In [1]:
import torch
from ternadecov.simulator import *
from ternadecov.time_deconv import *
from ternadecov.evaluation import *

# Configure

In [2]:
device = torch.device("cuda:0")
dtype = torch.float32
dtype_np = np.float32

# Load data

In [3]:
bulk_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_bulk.h5ad"
sc_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_sc.h5ad"

In [4]:
# with open(bulk_anndata_path, 'rb') as fh:
#     bulk_anndata  = anndata.read_h5ad(fh)
with open(sc_anndata_path, 'rb') as fh:
    sc_anndata = anndata.read_h5ad(fh)

# Simulate data

In [5]:
ebov_sc_dataset = SingleCellDataset(sc_anndata, sc_celltype_col="Subclustering_reduced", dtype_np=dtype_np, dtype = dtype, device=device)

In [6]:
# trajectory generating function
trajectory_type = 'periodic'
n_sample_range = list(range(100,1000,100)) 
n_sample_range

[100, 200, 300, 400, 500, 600, 700, 800, 900]

In [7]:
# Use a single trajectory for all iterations
trajectory_coef = sample_trajectories(
    type = trajectory_type,
    num_cell_types = ebov_sc_dataset.w_hat_gc.shape[1]
)

In [8]:
from typing import Dict

In [13]:
n_iters = 5_000

n_samples = []
l1_error_norm = []
fit_time = []

for n in n_sample_range:
    eval_result = evaluate_with_trajectory(
        sc_dataset = ebov_sc_dataset, 
        n_samples = n, 
        trajectory_type = trajectory_type, 
        trajectory_coef = trajectory_coef,
        n_iters = n_iters,
        dtype_np = dtype_np,
        dtype = dtype,
        device = device,
    )
    
    n_samples.append(eval_result['n_samples'])
    l1_error_norm.append(eval_result['l1_error_norm'])
    fit_time.append(eval_result['fit_time'])
    
evaluation_resuts_df = pd.DataFrame({'n_samples':n_samples, 'l1_error_norm': list(x.item() for x in l1_error_norm), 'fit_time': fit_time})

/home/nbarkas/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


KeyboardInterrupt: 

In [11]:
evaluation_resuts_df = pd.DataFrame({'n_samples':n_samples, 'l1_error_norm': list(x.item() for x in l1_error_norm), 'fit_time': fit_time})

In [12]:
evaluation_resuts_df

,n_samples,l1_error_norm,fit_time
0,100,1.299238,0.282878
1,200,1.299404,0.096314
2,300,1.299413,0.126664
3,400,1.299472,0.160437
4,500,1.299487,0.197816


In [ ]:
# Plot L1 loss
resuts_df 
resuts_df.plot(x='n',y='time')

In [ ]:
# Plot L1 loss
error_df = pd.DataFrame({'n':df_n, 'l1': list(x.item() for x in l1_error)})
error_df.plot(x='n',y='l1')